# NB02: Taxonomy Bridge and Functional Features

Build ENIGMA genus -> pangenome genus bridge and compute genus-level functional proxies from eggNOG COG categories.

Inputs:
- `../data/community_taxon_counts.tsv`

Outputs:
- `../data/taxon_bridge.tsv`
- `../data/taxon_functional_features.tsv`


In [1]:
from pathlib import Path
import re
import pandas as pd

DATA_DIR = Path('../data')
community = pd.read_csv(DATA_DIR / 'community_taxon_counts.tsv', sep='	')

print('Community rows:', len(community))
print('Unique ENIGMA genera:', community['genus'].nunique())

spark = get_spark_session()
print('Spark session ready')


Community rows: 41711
Unique ENIGMA genera: 1392


Spark session ready


In [2]:
def norm_genus(x: str) -> str:
    if pd.isna(x):
        return ''
    x = str(x).strip().lower()
    x = re.sub(r'^g__', '', x)
    x = re.sub(r'[^a-z0-9]+', '_', x)
    return x.strip('_')

community_genus = (
    community[['genus']]
    .dropna()
    .drop_duplicates()
    .assign(genus_norm=lambda d: d['genus'].map(norm_genus))
)
community_genus = community_genus[community_genus['genus_norm'] != '']
print('Community genus candidates:', len(community_genus))


Community genus candidates: 1392


In [3]:
species = spark.sql("""
SELECT DISTINCT *
FROM kbase_ke_pangenome.gtdb_species_clade
""").toPandas()

print('gtdb_species_clade columns:', species.columns.tolist())

species_col = None
for c in ['GTDB_species', 'gtdb_species', 'species', 'gtdb_species_name']:
    if c in species.columns:
        species_col = c
        break

if species_col is None:
    raise RuntimeError('Could not find species name column in gtdb_species_clade')
if 'gtdb_species_clade_id' not in species.columns:
    raise RuntimeError('Expected gtdb_species_clade_id in gtdb_species_clade')

species = species[['gtdb_species_clade_id', species_col]].rename(columns={species_col: 'species_name'})
species['genus_guess'] = (
    species['species_name']
    .astype(str)
    .str.replace('s__', '', regex=False)
    .str.split('_')
    .str[0]
)
species['genus_norm'] = species['genus_guess'].map(norm_genus)
species = species[species['genus_norm'] != ''].drop_duplicates()

print('Species rows with genus parsed:', len(species))


gtdb_species_clade columns: ['gtdb_species_clade_id', 'representative_genome_id', 'GTDB_species', 'GTDB_taxonomy', 'ANI_circumscription_radius', 'mean_intra_species_ANI', 'min_intra_species_ANI', 'mean_intra_species_AF', 'min_intra_species_AF', 'no_clustered_genomes_unfiltered', 'no_clustered_genomes_filtered']


Species rows with genus parsed: 27690


In [4]:
bridge = community_genus.merge(
    species[['genus_norm', 'gtdb_species_clade_id', 'species_name']],
    on='genus_norm',
    how='left'
)
bridge['mapping_tier'] = bridge['gtdb_species_clade_id'].apply(lambda v: 'genus_exact' if pd.notna(v) else 'unmapped')

mapped = bridge[bridge['mapping_tier'] == 'genus_exact'].copy()
print('Mapped genera:', mapped['genus'].nunique())
print('Unmapped genera:', (bridge['mapping_tier'] == 'unmapped').sum())

bridge.to_csv(DATA_DIR / 'taxon_bridge.tsv', sep='	', index=False)
print('Saved bridge table')


Mapped genera: 530
Unmapped genera: 862


Saved bridge table


In [5]:
mapped_clades = mapped[['genus', 'genus_norm', 'gtdb_species_clade_id']].drop_duplicates()
if mapped_clades.empty:
    raise RuntimeError('No mapped genera. Cannot compute functional features.')

spark.createDataFrame(mapped_clades[['genus_norm', 'gtdb_species_clade_id']]).createOrReplaceTempView('mapped_clades_tmp')

cog_counts = spark.sql("""
SELECT
  m.genus_norm,
  gc.gtdb_species_clade_id,
  e.COG_category,
  COUNT(*) AS n_annotations
FROM kbase_ke_pangenome.gene_cluster gc
JOIN mapped_clades_tmp m ON gc.gtdb_species_clade_id = m.gtdb_species_clade_id
JOIN kbase_ke_pangenome.eggnog_mapper_annotations e
  ON gc.gene_cluster_id = e.query_name
WHERE e.COG_category IS NOT NULL AND e.COG_category != ''
GROUP BY m.genus_norm, gc.gtdb_species_clade_id, e.COG_category
""").toPandas()

print('COG summary rows:', len(cog_counts))


COG summary rows: 477406


In [6]:
if cog_counts.empty:
    feats = pd.DataFrame(columns=['genus', 'genus_norm', 'feature_name', 'feature_value'])
else:
    cog_counts['n_annotations'] = pd.to_numeric(cog_counts['n_annotations'], errors='coerce').fillna(0)
    genus_tot = cog_counts.groupby('genus_norm', as_index=False)['n_annotations'].sum().rename(columns={'n_annotations':'total_ann'})

    def frac_for(letter):
        mask = cog_counts['COG_category'].astype(str).str.contains(letter, regex=False)
        tmp = cog_counts[mask].groupby('genus_norm', as_index=False)['n_annotations'].sum().rename(columns={'n_annotations':f'{letter}_ann'})
        out = genus_tot.merge(tmp, on='genus_norm', how='left').fillna(0)
        out[f'cog_{letter.lower()}_fraction'] = out[f'{letter}_ann'] / out['total_ann'].replace(0, pd.NA)
        return out[['genus_norm', f'cog_{letter.lower()}_fraction']]

    defense = frac_for('V').rename(columns={'cog_v_fraction': 'cog_defense_fraction'})
    mobilome = frac_for('X').rename(columns={'cog_x_fraction': 'cog_mobilome_fraction'})

    met_letters = ['C','E','G','H','I','P']
    mask_met = cog_counts['COG_category'].astype(str).apply(lambda s: any(ch in s for ch in met_letters))
    met = cog_counts[mask_met].groupby('genus_norm', as_index=False)['n_annotations'].sum().rename(columns={'n_annotations':'met_ann'})
    met = genus_tot.merge(met, on='genus_norm', how='left').fillna(0)
    met['cog_metabolism_fraction'] = met['met_ann'] / met['total_ann'].replace(0, pd.NA)
    met = met[['genus_norm', 'cog_metabolism_fraction']]

    wide = defense.merge(mobilome, on='genus_norm', how='outer').merge(met, on='genus_norm', how='outer').fillna(0)
    genus_lookup = mapped[['genus', 'genus_norm']].drop_duplicates()
    wide = genus_lookup.merge(wide, on='genus_norm', how='right')
    feats = wide.melt(id_vars=['genus', 'genus_norm'], var_name='feature_name', value_name='feature_value')

feats.to_csv(DATA_DIR / 'taxon_functional_features.tsv', sep='	', index=False)
print('Saved functional feature table')
print(feats.head().to_string(index=False))


Saved functional feature table
          genus      genus_norm         feature_name  feature_value
          67-14           67_14 cog_defense_fraction       0.015980
    Abiotrophia     abiotrophia cog_defense_fraction       0.040178
Acetoanaerobium acetoanaerobium cog_defense_fraction       0.022270
 Acetobacterium  acetobacterium cog_defense_fraction       0.027064
  Achromobacter   achromobacter cog_defense_fraction       0.011607
